In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report

In [5]:
ndf = pd.read_csv("new dataset.csv")
ndf

,Unnamed: 0,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,diff_org,diff_dest,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,0,9839.64,170136.00,160296.36,0.00,0.00,0,9839.64,0.00,0,0,1,0
1,1,1864.28,21249.00,19384.72,0.00,0.00,0,1864.28,0.00,0,0,1,0
2,2,181.00,181.00,0.00,0.00,0.00,1,181.00,0.00,0,0,0,1
3,3,181.00,181.00,0.00,21182.00,0.00,1,181.00,-21182.00,1,0,0,0
4,4,11668.14,41554.00,29885.86,0.00,0.00,0,11668.14,0.00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,6362615,339682.13,339682.13,0.00,0.00,339682.13,1,339682.13,339682.13,1,0,0,0
6362616,6362616,6311409.28,6311409.28,0.00,0.00,0.00,1,6311409.28,0.00,0,0,0,1
6362617,6362617,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,6311409.28,6311409.27,1,0,0,0
6362618,6362618,850002.52,850002.52,0.00,0.00,0.00,1,850002.52,0.00,0,0,0,1


In [6]:
ndf.drop("Unnamed: 0",axis = 1,inplace = True)

In [7]:
X = ndf.drop("isFraud",axis = 1)
y = ndf["isFraud"]

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state = 42)

In [10]:
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

In [18]:
model = xgb.XGBClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    eval_metric="logloss",
    n_jobs=-1,
    use_label_encoder=False,
)

In [19]:
model.fit(X_train, y_train)

c:\Users\hp\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:51:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=600, n_jobs=-1,
              num_parallel_tree=None, ...)

In [21]:
y_proba = model.predict_proba(X_test)[:, 1]

y_pred = (y_proba > 0.9).astype(int)

print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     1.0000    0.9991    0.9995   1270881
           1     0.5827    0.9732    0.7290      1643

    accuracy                         0.9991   1272524
   macro avg     0.7913    0.9862    0.8643   1272524
weighted avg     0.9994    0.9991    0.9992   1272524



In [ ]:
import joblib
joblib.dump(model,"fraud_detections.pkl")

['fraud_detection.pkl']